In [1]:
### Assumption : Confirmed_cases.csv exists in the working directory


from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from datetime import datetime
import collections

In [2]:
URL = "https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
br = webdriver.Chrome(executable_path=r"C:\Users/qasim/Downloads/chromedriver_win32_v80/chromedriver.exe",options=chrome_options)
br.get(URL)
time.sleep(3)

In [4]:
root = br.find_element_by_id("ember6")

In [5]:
data = root.text.split("\n")[2:]

In [6]:
end = data.index("Last Updated at (M/D/YYYY)")

In [7]:
total_cases = data[1]

In [8]:
start = 1 + data.index("Confirmed Cases by Country/Region/Sovereignty")

In [9]:
data = data[start:end]

In [10]:
total_cases = int(total_cases.replace(",",""))

In [11]:
TIME = str(datetime.now())

In [12]:
case = {}

In [13]:
for x in data:
    x = x.split(" ")
    country = " ".join(x[1:]).replace("*", "")
    cases = int(x[0].replace(",",""))
    case[country] = cases

In [14]:
case = collections.OrderedDict(sorted(case.items()))

In [15]:
df = pd.read_csv("Confirmed_case.csv")

In [16]:
df_temp = pd.DataFrame.from_dict(case, orient="index", columns=[':'.join(TIME.split(":")[:-1])]).reset_index()

In [402]:
del df_temp["index"]

In [403]:
df_cum = [df, df_temp]

In [404]:
df_cum = pd.concat(df_cum, axis=1)

In [406]:
df_cum.head()

,Country,2020-03-18 04:00,2020-03-18 04:02
0,Afghanistan,22,22
1,Albania,55,55
2,Algeria,60,60
3,Andorra,39,39
4,Antigua and Barbuda,1,1


In [412]:
df_cum.to_csv("Confirmed_case.csv", index=False)